In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score

from .utils._logger import logger

In [ ]:
from pandas import DataFrame


def load_and_inspect_data(file_path: str) -> DataFrame | None:
    """
    Loads the weather forecast data, performs initial inspection, and handles data type conversions.

    Args:
        file_path (str): The path to the cleaned weather forecast CSV file.

    Returns:
        pandas.DataFrame: The processed weather data DataFrame.
    """
    try:
        df: DataFrame = pd.read_csv(file_path, parse_dates=["date"])
        df = df[["date", "T", "rh", "wv"]]
        logger.info(f"Data loaded successfully from {file_path}")
        return df
    except FileNotFoundError:
        logger.error(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        logger.error(f"An error occurred while loading the data: {e}")
        return None

In [ ]:
def forecast_temperature(df):
    """
    Performs time series forecasting of temperature using the Prophet library.

    Args:
        df (pandas.DataFrame): The weather data DataFrame with "ds" (date) and "y" (temperature) columns.
    Returns:
        pandas.DataFrame: The forecasted temperature data.
    """
    df_prophet = df.reset_index().rename(columns={"date": "ds", "T": "y"})

    # Train split
    train_size = int(len(df_prophet) * 0.8)
    p_train, p_test = df_prophet.iloc[:train_size], df_prophet.iloc[train_size:]

    # Fit model
    model = Prophet()
    model.fit(p_train)

    # Make future dataframe
    future = model.make_future_dataframe(periods=len(p_test), freq="10min")
    forecast = model.predict(future)

    # Extract the forecast
    predicted = forecast[["ds", "yhat"]].set_index("ds").loc[p_test["ds"]]

    mse: float = mean_squared_error(p_test["y"], predicted["yhat"])
    r2: float = r2_score(p_test["y"], predicted["yhat"])
    print(f"Test MSE: {mse:.3f}")
    print(f"Test R²: {r2:.3f}")

    model.plot(forecast)
    plt.title("Total Sales Forecast")
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.show()

    model.plot_components(forecast)
    plt.show()

    return df, predicted

In [ ]:
df = load_and_inspect_data("path")

In [ ]:
df, predicted = forecast_temperature(df)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df["date"], df["T"], label="Actual")
plt.plot(predicted.index, predicted["yhat"], color="red", label="Forecast")
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.title("Prophet Forecast")
plt.legend()
plt.show()